In [1]:
from pathlib import Path
import pandas as pd
csv_files = list((Path().glob('*.csv')))
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    split = str(file).split('_')
    architecture = split[0]
    tokenizer = split[-2]
    random_weights = False
    if('RANDOM' in split):
        random_weights = True
        
    df.insert(0, 'random_weights',random_weights)
    df.insert(0, 'tokenizer',tokenizer)
    df.insert(0, 'architecture',architecture)
    
    df['file_name'] = file
    dfs.append(df)
    # print(df.head())

main_df = pd.concat(dfs, ignore_index=True)

In [2]:
dsets = list(main_df.dataset.unique())

In [3]:
# main_df.head()

In [4]:
dfs = []
for dataset_index in range(6):
    df = main_df[
        # (main_df['random_weights']==True) & 
        # (main_df['architecture']=='DEBERTA') & 
        (main_df['dataset']==dsets[dataset_index])
    ][['architecture','dataset','tokenizer','random_weights','test_acc', 'test_f1', 'test_auroc_weighted','test_pr_auc']]\
    .sort_values('tokenizer')\
    .style\
    .hide(axis='index')\
    .format({"dataset": lambda x:x.lower()})\
    .background_gradient(subset=['test_acc', 'test_f1', 'test_auroc_weighted','test_pr_auc'], cmap='Blues')
    # .highlight_max(subset=['test_auroc_weighted'],color='lightgreen')\
    # .highlight_min(subset=['test_auroc_weighted'],color='#cd4f39')\
    dfs.append(df)
    # main_df.iloc[main_df.groupby(['dataset','tokenizer'])['test_auroc_weighted'].idxmax()]
    # main_df.iloc[main_df.groupby('dataset')['test_acc'].idxmax()]
    





In [5]:
dfs[0]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,False,0.924600,0.925936,0.978707,0.978504
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.900920,0.900606,0.963096,0.960968
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,False,0.924600,0.925936,0.978707,0.978504
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.900920,0.900606,0.963096,0.960968
DEBERTA,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.901200,0.901492,0.963793,0.961481
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.899560,0.901425,0.963367,0.961143
BERT,demo_coding_vs_intergenomic_seqs,Kmer7tokenizer,True,0.898840,0.901584,0.963771,0.962069
DEBERTA,demo_coding_vs_intergenomic_seqs,Kmer7tokenizer,True,0.901000,0.899452,0.963838,0.962015
DEBERTA,demo_coding_vs_intergenomic_seqs,Kmer8tokenizer,True,0.895320,0.895215,0.960163,0.959543
DEBERTA,demo_coding_vs_intergenomic_seqs,Kmer8tokenizer,False,0.911480,0.908926,0.971529,0.972589


In [6]:
dfs[1]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,demo_human_or_worm,DNABERTtokenizer,False,0.965800,0.965733,0.995246,0.995429
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.953480,0.952710,0.991099,0.991605
BERT,demo_human_or_worm,DNABERTtokenizer,False,0.965800,0.965733,0.995246,0.995429
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.953480,0.952710,0.991099,0.991605
DEBERTA,demo_human_or_worm,DNABERTtokenizer,True,0.954600,0.954912,0.991370,0.991745
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.954080,0.953925,0.990508,0.991099
BERT,demo_human_or_worm,Kmer7tokenizer,True,0.955720,0.955597,0.991495,0.991811
DEBERTA,demo_human_or_worm,Kmer7tokenizer,True,0.955400,0.955622,0.991569,0.991967
DEBERTA,demo_human_or_worm,Kmer8tokenizer,True,0.948320,0.948567,0.989329,0.989908
DEBERTA,demo_human_or_worm,Kmer8tokenizer,False,0.958080,0.958557,0.993469,0.993667


In [7]:
dfs[2]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_enhancers_cohn,DNABERTtokenizer,False,0.741940,0.749476,0.828015,0.818516
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.726540,0.733819,0.800259,0.789622
BERT,human_enhancers_cohn,DNABERTtokenizer,False,0.741940,0.749476,0.828015,0.818516
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.726540,0.733819,0.800259,0.789622
DEBERTA,human_enhancers_cohn,DNABERTtokenizer,True,0.730282,0.729191,0.804544,0.799972
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.728411,0.709558,0.806360,0.800327
BERT,human_enhancers_cohn,Kmer7tokenizer,True,0.727979,0.728370,0.803234,0.793617
DEBERTA,human_enhancers_cohn,Kmer7tokenizer,True,0.720927,0.724456,0.797471,0.788885
DEBERTA,human_enhancers_cohn,Kmer8tokenizer,True,0.705959,0.673381,0.791136,0.783248
DEBERTA,human_enhancers_cohn,Kmer8tokenizer,False,0.722798,0.750065,0.811191,0.796354


In [8]:
dfs[3]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_enhancers_ensembl,DNABERTtokenizer,False,0.866096,0.865867,0.941497,0.942554
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.799871,0.811748,0.889409,0.892675
BERT,human_enhancers_ensembl,DNABERTtokenizer,False,0.866096,0.865867,0.941497,0.942554
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.799871,0.811748,0.889409,0.892675
DEBERTA,human_enhancers_ensembl,DNABERTtokenizer,True,0.812851,0.810848,0.893935,0.895679
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.805392,0.798906,0.882320,0.884570
BERT,human_enhancers_ensembl,Kmer7tokenizer,True,0.796384,0.787175,0.872875,0.870909
DEBERTA,human_enhancers_ensembl,Kmer7tokenizer,True,0.795221,0.790319,0.870675,0.870248
DEBERTA,human_enhancers_ensembl,Kmer8tokenizer,True,0.793736,0.797026,0.866356,0.861359
DEBERTA,human_enhancers_ensembl,Kmer8tokenizer,False,0.834420,0.842516,0.918401,0.915678


In [9]:
dfs[4]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_nontata_promoters,DNABERTtokenizer,False,0.828979,0.815125,0.926020,0.951453
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.823998,0.809398,0.920357,0.947520
BERT,human_nontata_promoters,DNABERTtokenizer,False,0.828979,0.815125,0.926020,0.951453
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.823998,0.809398,0.920357,0.947520
DEBERTA,human_nontata_promoters,DNABERTtokenizer,True,0.835510,0.824225,0.924087,0.950049
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.831636,0.819552,0.921892,0.948723
BERT,human_nontata_promoters,Kmer7tokenizer,True,0.858424,0.851916,0.944867,0.963659
DEBERTA,human_nontata_promoters,Kmer7tokenizer,True,0.871043,0.867146,0.946988,0.965614
DEBERTA,human_nontata_promoters,Kmer8tokenizer,True,0.892517,0.894353,0.958166,0.970961
DEBERTA,human_nontata_promoters,Kmer8tokenizer,False,0.912553,0.913755,0.966116,0.977803


In [10]:
dfs[5]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
BERT,human_ocr_ensembl,DNABERTtokenizer,False,0.768511,0.762344,0.850755,0.839602
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.714952,0.721322,0.788795,0.775356
BERT,human_ocr_ensembl,DNABERTtokenizer,False,0.768511,0.762344,0.850755,0.839602
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.714952,0.721322,0.788795,0.775356
DEBERTA,human_ocr_ensembl,DNABERTtokenizer,True,0.718156,0.729480,0.794750,0.782172
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.713836,0.727302,0.789594,0.774474
BERT,human_ocr_ensembl,Kmer7tokenizer,True,0.726797,0.740552,0.800538,0.784340
DEBERTA,human_ocr_ensembl,Kmer7tokenizer,True,0.723764,0.725624,0.799970,0.786248
DEBERTA,human_ocr_ensembl,Kmer8tokenizer,True,0.719072,0.728043,0.791334,0.777627
DEBERTA,human_ocr_ensembl,Kmer8tokenizer,False,0.755579,0.779882,0.845863,0.833512


In [11]:
from genomic_benchmarks.data_check.info import list_datasets

/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [12]:
main_df.keys()

Index(['architecture', 'tokenizer', 'random_weights', 'dataset', 'test_acc',
       'test_f1', 'test_loss', 'test_precision', 'test_recall',
       'test_auroc_macro', 'test_auroc_weighted', 'test_pr_auc',
       'min_valid_loss_epoch', 'min_valid_loss_log', 'file_name'],
      dtype='object')

In [13]:
all_datasets = list_datasets()
all_datasets.remove('dummy_mouse_enhancers_ensembl')
all_datasets

['human_enhancers_cohn',
 'human_nontata_promoters',
 'human_ensembl_regulatory',
 'drosophila_enhancers_stark',
 'demo_human_or_worm',
 'demo_coding_vs_intergenomic_seqs',
 'human_enhancers_ensembl',
 'human_ocr_ensembl']

In [14]:
categories = ['dataset', 'architecture', 'tokenizer', 'random_weights']
exp_options = {}
for category in categories:
    exp_options[category] = []

for category in categories:
    exp_options[category].extend(main_df[category].unique())
exp_options

{'dataset': ['demo_coding_vs_intergenomic_seqs',
  'demo_human_or_worm',
  'human_enhancers_cohn',
  'human_enhancers_ensembl',
  'human_nontata_promoters',
  'human_ocr_ensembl',
  'drosophila_enhancers_stark'],
 'architecture': ['BERT', 'DEBERTA', 'NAME'],
 'tokenizer': ['DNABERTtokenizer',
  'Sentencepiece10tokenizer',
  'Sentencepiece30tokenizer',
  'Kmer8tokenizer',
  'Kmer7tokenizer',
  'NAME'],
 'random_weights': [False, True]}

In [20]:
all_combinations = []
actual_combinations = []

exptected_len = len(exp_options['dataset']) * len(exp_options['architecture']) * len(exp_options['tokenizer']) * len(exp_options['random_weights'])
print('exptected_len ', exptected_len)

for dataset in exp_options['dataset']:
    for architecture in exp_options['architecture']:
        for tokenizer in exp_options['tokenizer']:
            for random_weights in exp_options['random_weights']:
                all_combinations.append(tuple((dataset, architecture, tokenizer, random_weights)))
        
print('all_combinations ', len(set(all_combinations)))

all_combinations_unique = set(all_combinations)
print('all_combinations_unique ', len(set(all_combinations_unique)), ' should equal all_combinations')

main_df.reset_index()
for index, row in main_df.iterrows():
    row_dic = {}    
    for cat in categories:
        row_dic[cat] = row[cat]
        
    actual_combinations.append(tuple((row['dataset'], row['architecture'], row['tokenizer'], row['random_weights'])))

actual_combinations_unique = set(actual_combinations)

print('actual_combinations ', len(actual_combinations))
print('actual_combinations_unique ', len(actual_combinations_unique))

missing_combinations_unique = all_combinations_unique.difference(actual_combinations_unique)

print('missing_combinations_unique ', len(missing_combinations_unique))

missing_combinations_unique = pd.DataFrame(list(missing_combinations_unique), columns=categories)
missing_combinations_unique


exptected_len  252
all_combinations  252
all_combinations_unique  252  should equal all_combinations
actual_combinations  116
actual_combinations_unique  86
missing_combinations_unique  166


,dataset,architecture,tokenizer,random_weights
0,demo_human_or_worm,NAME,Kmer8tokenizer,True
1,human_nontata_promoters,NAME,DNABERTtokenizer,False
2,human_ocr_ensembl,DEBERTA,NAME,True
3,drosophila_enhancers_stark,DEBERTA,Sentencepiece10tokenizer,False
4,human_ocr_ensembl,BERT,Sentencepiece30tokenizer,False
...,...,...,...,...
161,human_enhancers_cohn,BERT,Sentencepiece30tokenizer,False
162,human_nontata_promoters,DEBERTA,Kmer7tokenizer,False
163,human_nontata_promoters,BERT,Sentencepiece30tokenizer,False
164,drosophila_enhancers_stark,BERT,Kmer8tokenizer,True


In [22]:
missing_combinations_unique[missing_combinations_unique['dataset'] == 'drosophila_enhancers_stark']

,dataset,architecture,tokenizer,random_weights
3,drosophila_enhancers_stark,DEBERTA,Sentencepiece10tokenizer,False
10,drosophila_enhancers_stark,NAME,Kmer7tokenizer,True
24,drosophila_enhancers_stark,BERT,Sentencepiece30tokenizer,True
29,drosophila_enhancers_stark,NAME,Kmer8tokenizer,True
31,drosophila_enhancers_stark,DEBERTA,DNABERTtokenizer,False
34,drosophila_enhancers_stark,BERT,Kmer7tokenizer,False
40,drosophila_enhancers_stark,NAME,Sentencepiece10tokenizer,True
42,drosophila_enhancers_stark,BERT,DNABERTtokenizer,False
48,drosophila_enhancers_stark,DEBERTA,NAME,False
58,drosophila_enhancers_stark,NAME,DNABERTtokenizer,True
